# Indicator Speed Test

This Notebook shows the speed of each indicator _with_ and _without_ TA Lib* Results may vary if ```vectorbt``` or ```numba``` is installed.
* These values are based on a M1 Macbook with 16GB Memory.

In [ ]:
from importlib.util import find_spec

from numpy import version as numpy_version
from pandas import DataFrame, IndexSlice, concat, read_csv, __version__
from pandas import IndexSlice as idx
import polars_ti as ti

print("Package Versions:")
print(f"Pandas v{__version__}")
print(f"Polars TI v{ti.version}")

has_numba = find_spec("numba") is not None
if has_numba:
    from numba import __version__ as numba_version

    print(f"Numba v{numba_version}")

if find_spec("talib") is not None:
    from talib import __version__ as tal_version

    print(f"talib v{tal_version}")

from pandas import read_csv
from pandas import DatetimeIndex as dti

%matplotlib inline

### Fetch Sample Data

In [ ]:
df = DataFrame()
_df = df.ti.ticker("SPY", period="5y", timed=True)

In [ ]:
df = _df.copy()
df.shape

### If ```numba``` installed, prep @njit functions
* Some indicators like **ha** have numba/njit versions, so they need to be compiled by have a prerun. In the performance tests after this, **ha** and other numba/njit versions will be quicker also.

In [ ]:
if has_numba:
    ti.speed_test(df.iloc[-150:], talib=False, silent=True)

## Performance **without** TA Lib

In [ ]:
pti_speedsdf, pti_statsdf = ti.speed_test(
    df, top=10, talib=False, stats=True, gradient=True, verbose=True
)

In [ ]:
pti_speedsdf

In [ ]:
pti_statsdf

## Performance **with** TA Lib

In [ ]:
tal_speedsdf, tal_statsdf = ti.speed_test(
    df, top=10, talib=True, stats=True, gradient=True, verbose=True
)

In [ ]:
tal_speedsdf

In [ ]:
tal_statsdf

# Comparisons

In [ ]:
print(df.shape)
compdf = concat([tal_statsdf, pti_statsdf], keys=["TA Lib", "Polars TI"], axis=1).T
compdf

In [ ]:
diffdf = (tal_statsdf - pti_statsdf).abs().T
diffdf.columns.name = "Differences"
diffdf

## Speed Comparison (ms)

In [ ]:
taldf = ti.speed_test(df, places=10, talib=True, silent=True)["ms"]
ptidf = ti.speed_test(df, places=10, talib=False, silent=True)["ms"]

In [ ]:
scdf = DataFrame({"TAL": taldf, "PTI": ptidf, "diff": taldf - ptidf})
scdf.round(decimals=3)
scdf.index.name = "Indicator (ms)"

In [ ]:
scdf.sort_values(by=["TAL", "diff"], ascending=False, na_position="first", inplace=True)
scdf.style.background_gradient("autumn_r")